In [1]:
import hashlib
import time
from typing import List, Dict, Any
import json
import random
import string

In [2]:
class MerkleTree:
    def __init__(self, transactions: List[str]):
        self.transactions = transactions
        self.levels = []
        if transactions:
            self.build_tree()

    def hash(self, data: str) -> str:
        return hashlib.sha256(data.encode()).hexdigest()

    def build_tree(self):
        current_level = [self.hash(tx) for tx in self.transactions]
        self.levels.append(current_level)

        while len(current_level) > 1:
            next_level = []
            for i in range(0, len(current_level), 2):
                left = current_level[i]
                right = current_level[i+1] if i+1 < len(current_level) else left
                combined = self.hash(left + right)
                next_level.append(combined)
            current_level = next_level
            self.levels.append(current_level)

    def get_root(self) -> str:
        return self.levels[-1][0] if self.levels else None

In [3]:
class Transaction:
    def __init__(self, keyword: str, value: int, metadata: Dict[str, Any]):
        self.keyword = keyword
        self.value = value
        self.metadata = metadata  # additional info
        self.timestamp = int(time.time())

    def serialize(self):
        return json.dumps({
            "keyword": self.keyword,
            "value": self.value,
            "metadata": self.metadata,
            "timestamp": self.timestamp
        })

In [4]:
class Block:
    def __init__(self, index: int, prev_hash: str, transactions: List[Transaction]):
        self.index = index
        self.prev_hash = prev_hash
        self.timestamp = int(time.time())
        self.transactions = transactions

        serialized = [tx.serialize() for tx in transactions]
        self.merkle_tree = MerkleTree(serialized)
        self.merkle_root = self.merkle_tree.get_root()
        self.hash = self.compute_hash()

    def compute_hash(self) -> str:
        block_string = json.dumps({
            "index": self.index,
            "prev_hash": self.prev_hash,
            "timestamp": self.timestamp,
            "merkle_root": self.merkle_root
        })
        return hashlib.sha256(block_string.encode()).hexdigest()

In [5]:
class Blockchain:
    def __init__(self):
        genesis = Block(0, "0", [])
        self.chain = [genesis]

    def add_block(self, transactions: List[Transaction]):
        prev_hash = self.chain[-1].hash
        new_block = Block(len(self.chain), prev_hash, transactions)
        self.chain.append(new_block)

    def get_all_transactions(self):
        txs = []
        for block in self.chain:
            txs.extend(block.transactions)
        return txs

In [6]:
class QueryEngine:
    def __init__(self, blockchain: Blockchain):
        self.blockchain = blockchain

    # 🔹 Latest-K (Freshness Query)
    def latest_k(self, k: int):
        txs = sorted(self.blockchain.get_all_transactions(), key=lambda t: t.timestamp, reverse=True)
        return txs[:k]

    # 🔹 Keyword Match Query
    def search_keyword(self, keyword: str):
        return [tx for tx in self.blockchain.get_all_transactions() if tx.keyword == keyword]

    # 🔹 Range Query (Value-based)
    def range_query(self, min_v: int, max_v: int):
        return [tx for tx in self.blockchain.get_all_transactions() if min_v <= tx.value <= max_v]

    # 🔹 Aggregate Query (Sum)
    def aggregate_sum(self):
        return sum(tx.value for tx in self.blockchain.get_all_transactions())

In [7]:
def random_keyword():
    return random.choice(["food", "electronics", "grocery", "fashion", "fuel"])

def generate_transactions(n=10):
    txs = []
    for _ in range(n):
        keyword = random_keyword()
        value = random.randint(10, 500)
        metadata = {"uid": ''.join(random.choices(string.ascii_letters, k=5))}
        txs.append(Transaction(keyword, value, metadata))
    return txs

In [8]:
blockchain = Blockchain()

# create 5 blocks
for _ in range(5):
    txs = generate_transactions(8)
    blockchain.add_block(txs)

In [9]:
engine = QueryEngine(blockchain)

# Latest-K
latest = engine.latest_k(5)
print("Latest 5 Transactions:")
for tx in latest:
    print(tx.serialize())

# Keyword Query
print("\nTransactions with keyword = 'food':")
for tx in engine.search_keyword("food"):
    print(tx.serialize())

# Range Query
print("\nTransactions with values between 100 and 300:")
for tx in engine.range_query(100, 300):
    print(tx.serialize())

# Aggregate Query
total_value = engine.aggregate_sum()
print("\nTotal Value of All Transactions:", total_value)

Latest 5 Transactions:
{"keyword": "grocery", "value": 249, "metadata": {"uid": "pKCvp"}, "timestamp": 1765373274}
{"keyword": "food", "value": 387, "metadata": {"uid": "yXuTr"}, "timestamp": 1765373274}
{"keyword": "food", "value": 281, "metadata": {"uid": "oCNej"}, "timestamp": 1765373274}
{"keyword": "fashion", "value": 27, "metadata": {"uid": "EaDsN"}, "timestamp": 1765373274}
{"keyword": "fashion", "value": 461, "metadata": {"uid": "mGBUN"}, "timestamp": 1765373274}

Transactions with keyword = 'food':
{"keyword": "food", "value": 387, "metadata": {"uid": "yXuTr"}, "timestamp": 1765373274}
{"keyword": "food", "value": 281, "metadata": {"uid": "oCNej"}, "timestamp": 1765373274}
{"keyword": "food", "value": 70, "metadata": {"uid": "JIUNo"}, "timestamp": 1765373274}
{"keyword": "food", "value": 374, "metadata": {"uid": "muiTC"}, "timestamp": 1765373274}
{"keyword": "food", "value": 303, "metadata": {"uid": "ZwDoQ"}, "timestamp": 1765373274}
{"keyword": "food", "value": 431, "metadata

In [10]:
def verify_block_integrity(block: Block):
    recalculated_hash = block.compute_hash()
    return recalculated_hash == block.hash

In [11]:
print("\nVerifying Blockchain Integrity:")
for block in blockchain.chain:
    print(f"Block {block.index} integrity:", verify_block_integrity(block))


Verifying Blockchain Integrity:
Block 0 integrity: True
Block 1 integrity: True
Block 2 integrity: True
Block 3 integrity: True
Block 4 integrity: True
Block 5 integrity: True


In [12]:
#query1 Find All Transactions with Keyword "electronics"
results = engine.search_keyword("electronics")
for tx in results:
    print(tx.serialize())

{"keyword": "electronics", "value": 73, "metadata": {"uid": "QgLUq"}, "timestamp": 1765373274}
{"keyword": "electronics", "value": 240, "metadata": {"uid": "LUtwX"}, "timestamp": 1765373274}
{"keyword": "electronics", "value": 251, "metadata": {"uid": "KwwZQ"}, "timestamp": 1765373274}


In [13]:
#query2 Get All Transactions with Value Between 150 and 300
results = engine.range_query(150, 300)
for tx in results:
    print(tx.serialize())

{"keyword": "grocery", "value": 249, "metadata": {"uid": "pKCvp"}, "timestamp": 1765373274}
{"keyword": "food", "value": 281, "metadata": {"uid": "oCNej"}, "timestamp": 1765373274}
{"keyword": "fashion", "value": 243, "metadata": {"uid": "Kvpih"}, "timestamp": 1765373274}
{"keyword": "grocery", "value": 272, "metadata": {"uid": "ADyQQ"}, "timestamp": 1765373274}
{"keyword": "fashion", "value": 199, "metadata": {"uid": "yuWvD"}, "timestamp": 1765373274}
{"keyword": "grocery", "value": 282, "metadata": {"uid": "pQIbm"}, "timestamp": 1765373274}
{"keyword": "fuel", "value": 189, "metadata": {"uid": "KzMIo"}, "timestamp": 1765373274}
{"keyword": "fuel", "value": 159, "metadata": {"uid": "MBCEG"}, "timestamp": 1765373274}
{"keyword": "fashion", "value": 218, "metadata": {"uid": "xyreX"}, "timestamp": 1765373274}
{"keyword": "fuel", "value": 289, "metadata": {"uid": "ckvqI"}, "timestamp": 1765373274}
{"keyword": "fashion", "value": 287, "metadata": {"uid": "kURbC"}, "timestamp": 1765373274}


In [14]:
#query3 Total Value of All Transactions
total = engine.aggregate_sum()
print("Total Value of All Transactions:", total)

Total Value of All Transactions: 10581


In [15]:
#query4 Verify Integrity of a Block
integrity = verify_block_integrity(blockchain.chain[2])
print("Block 2 integrity:", integrity)

Block 2 integrity: True


In [16]:
#query5 Top 3 most expensive transactions
txs = blockchain.get_all_transactions()
top3 = sorted(txs, key=lambda t: t.value, reverse=True)[:3]

for tx in top3:
    print(tx.serialize())

{"keyword": "grocery", "value": 477, "metadata": {"uid": "BxSAi"}, "timestamp": 1765373274}
{"keyword": "grocery", "value": 467, "metadata": {"uid": "wwoAU"}, "timestamp": 1765373274}
{"keyword": "fashion", "value": 461, "metadata": {"uid": "mGBUN"}, "timestamp": 1765373274}
